In [1]:
from modules.transformer import Transformer
from options.base_options import BaseOptions
from models.Transform2D import Transform2D
from datasets.shape_net import ShapeNet
import torch
%load_ext autoreload
%autoreload 2

In [2]:
dataset = ShapeNet(cat="airplane")
len(dataset)
model = Transform2D()

In [6]:
images = dataset[0]['images']
voxels = dataset[0]['voxels']
voxels.shape
data = dataset[0]
#model.forward(data)
voxels.shape

(1, 32, 32, 32)

In [5]:
model(data)

c:\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


> c:\users\youssef\repos\tum\ss23\3dml\exercises\project\src\models\transform2d.py(39)forward()
     37 
     38         import pdb;pdb.set_trace()
---> 39         return x
     40 
     41 

torch.Size([23, 32, 32, 32])


In [ ]:

from utils.model_utils import summarize_model
total_params = sum(p.numel() for p in model.transformer.parameters())
total_params

In [ ]:

x = torch.ones((3,5,5,5,2)) 

In [ ]:
from einops import rearrange
x[0][0][0][0]
y = rearrange(x, 'bs c1 c2 c3 c4 -> bs c4 c1 c2 c3 ')
y.shape

In [ ]:
y
sfy = torch.nn.Softmax(dim=1)
sfx = torch.nn.Softmax(dim=-1)
y2 = sfy(y)
x2 = sfx(x)

In [ ]:
y2
x2
y3 = rearrange(y2,'bs c4 c1 c2 c3 ->bs c1 c2 c3 c4')

In [ ]:
y3==x2


In [ ]:
n_deconvfilter = [128, 128, 128, 64, 32, 2]
filter_shape = (n_deconvfilter[1], 3, 3, 3)
filter_shape = (n_deconvfilter[3], 3, 3, 3)
padding = [0, int((filter_shape[1] - 1) / 2), 0, int((filter_shape[2] - 1) / 2),
                             int((filter_shape[3] - 1) / 2) ] 

In [ ]:
padding